<a href="https://colab.research.google.com/github/maitreyeejadhav/Fake-News-Detection/blob/main/svm_dataset_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix # for confusion matrix

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
true_data=pd.read_csv('True.csv')
fake_data=pd.read_csv('Fake.csv')

print(true_data[0:10])
fake_data[0:4]

                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   
5  White House, Congress prepare for talks on spe...   
6  Trump says Russia probe will be fair, but time...   
7  Factbox: Trump on Twitter (Dec 29) - Approval ...   
8         Trump on Twitter (Dec 28) - Global Warming   
9  Alabama official to certify Senator-elect Jone...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) -

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"


In [ ]:
print(true_data['text'][0])
true_data.columns

WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educati

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [ ]:
len(true_data)

21417

In [ ]:
len(fake_data)

23481

In [ ]:
fake_data['label'] = 1
true_data['label']=0
print(true_data.head())
fake_data.head()

                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  label  
0  December 31, 2017       0  
1  December 29, 2017       0  
2  December 31, 2017       0  
3  December 30, 2017       0  
4  December 29, 2017       0  


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
all_data=pd.concat([fake_data, true_data])
random_permutation = np.random.permutation(len(all_data))
all_data= all_data.iloc[random_permutation]
print(all_data.columns)
all_data.head()

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')


,title,text,subject,date,label
8777,Obama plan to stop tax inversions stirs U.S. b...,WASHINGTON (Reuters) - The Obama administratio...,politicsNews,"July 13, 2016",0
8155,Here’s A Simple How-To Guide For Shooting Dow...,The only thing more irritating to a woman than...,News,"February 12, 2016",1
12838,WOW! HOW 100% OF CLINTON CHARITY Went To…Thems...,,politics,"Oct 4, 2016",1
2069,Germany not first in line to send more troops ...,"ECKERNFOERDE, Germany (Reuters) - Germany will...",politicsNews,"August 22, 2017",0
17018,LAWLESS: MUSLIM VIOLENCE And Refusal To Assimi...,The European caliphate is in full swing First ...,Government News,"Oct 21, 2015",1


In [ ]:
filterd_data=all_data.loc[:, ['title', 'text', "subject", 'label']]
filterd_data.head()

,title,text,subject,label
8777,Obama plan to stop tax inversions stirs U.S. b...,WASHINGTON (Reuters) - The Obama administratio...,politicsNews,0
8155,Here’s A Simple How-To Guide For Shooting Dow...,The only thing more irritating to a woman than...,News,1
12838,WOW! HOW 100% OF CLINTON CHARITY Went To…Thems...,,politics,1
2069,Germany not first in line to send more troops ...,"ECKERNFOERDE, Germany (Reuters) - Germany will...",politicsNews,0
17018,LAWLESS: MUSLIM VIOLENCE And Refusal To Assimi...,The European caliphate is in full swing First ...,Government News,1


In [ ]:
filterd_data.isnull().sum()

title      0
text       0
subject    0
label      0
dtype: int64

In [ ]:
filterd_data['training_feature']=filterd_data['title']+' '+filterd_data['text']+' '+filterd_data['subject']
filterd_data.head()

,title,text,subject,label,training_feature
8777,Obama plan to stop tax inversions stirs U.S. b...,WASHINGTON (Reuters) - The Obama administratio...,politicsNews,0,Obama plan to stop tax inversions stirs U.S. b...
8155,Here’s A Simple How-To Guide For Shooting Dow...,The only thing more irritating to a woman than...,News,1,Here’s A Simple How-To Guide For Shooting Dow...
12838,WOW! HOW 100% OF CLINTON CHARITY Went To…Thems...,,politics,1,WOW! HOW 100% OF CLINTON CHARITY Went To…Thems...
2069,Germany not first in line to send more troops ...,"ECKERNFOERDE, Germany (Reuters) - Germany will...",politicsNews,0,Germany not first in line to send more troops ...
17018,LAWLESS: MUSLIM VIOLENCE And Refusal To Assimi...,The European caliphate is in full swing First ...,Government News,1,LAWLESS: MUSLIM VIOLENCE And Refusal To Assimi...


In [ ]:
X= filterd_data['training_feature'].values
y = filterd_data['label']

In [ ]:
l_X=filterd_data['training_feature'].values[0:1000]
l_Y= filterd_data['label'].values[0:1000]

In [ ]:
print(l_X.shape)
print(l_Y.shape)
type (l_X)

(1000,)
(1000,)


numpy.ndarray

In [ ]:
print(X[0:1])

['Obama plan to stop tax inversions stirs U.S. business concerns WASHINGTON (Reuters) - The Obama administration’s plan to prevent American companies from shifting their headquarters overseas to avoid U.S. taxes is coming under fire from companies and banks that say it would be costly and cumbersome. At issue are proposed Treasury regulations to combat “earnings stripping,” a key goal for companies that carry out tax-avoiding mergers known as “inversions” to reincorporate abroad, if only on paper, to cut their taxes. The practice effectively shifts taxable earnings from U.S. operations to the redomiciled former American parent as debt interest payments that are tax deductible in the United States and subject to a lower income tax rate overseas.  The Treasury Department is scheduled to hold a public hearing on the proposed changes on Thursday. The administration’s proposals, which could be finalized within months, have already dampened interest in global mergers. The proposals are backe

In [ ]:
vectorizer= TfidfVectorizer()
X=vectorizer.fit_transform(X)

In [ ]:
l_vectorizer= TfidfVectorizer()
l_X=l_vectorizer.fit_transform(l_X)

In [ ]:
print(type(X))
print(X.shape)
print(type(l_X))
print(l_X.shape)

<class 'scipy.sparse._csr.csr_matrix'>
(44898, 122516)
<class 'scipy.sparse._csr.csr_matrix'>
(1000, 22489)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [ ]:
l_X_train, l_X_test, l_Y_train, l_Y_test = train_test_split(l_X, l_Y, test_size = 0.2, random_state=42)

In [ ]:
X_train.shape

(35918, 122516)

model training


In [ ]:
from sklearn import svm

#Create a svm Classifier
model = svm.SVC(kernel='linear') # Linear Kernel
clf_poly=clf = svm.SVC(kernel='poly') # Polynomial kernel

In [ ]:
def stemming(content):   #stemming is a function here
  stemmed_content = re.sub('[^a-zA-z]',' ',content)       #removes everything that is not btw aA-zZ(all numbers and puntuations replaced by space)
  stemmed_content = stemmed_content.lower()                #converts all uc to lc to avoid problem during processing
  stemmed_content = stemmed_content.split()                #all words and text will be splitted and converted to list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
print(X_train.shape)
# X_train.iloc[0:1000].shape

(35918, 122516)


In [ ]:
#Train the model using the training sets
model.fit(l_X_train, l_Y_train)
# clf_poly.fit(X_train,Y_train)

SVC(kernel='linear')

In [ ]:
#Predict the response for test dataset
y_pred = model.predict(l_X_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print(accuracy_score(y_pred, l_Y_test))
print (precision_score(l_Y_test, y_pred,))
print (recall_score(l_Y_test, y_pred,))
print(f1_score(l_Y_test, y_pred,))
print(confusion_matrix(l_Y_test, y_pred,))

0.96
0.9811320754716981
0.9454545454545454
0.9629629629629629
[[ 88   2]
 [  6 104]]
